In [1]:
import pandas as pd
import numpy as np
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F

sys.path.append('../../../')

from configs.data_configs.rosbank import data_configs
from configs.model_configs.gen.rosbank import model_configs
from src.data_load.dataloader import create_data_loaders, create_test_loader

from src.models.TimeGan import TG

import torch
import numpy as np
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

In [ ]:
def random_generator(batch_size, z_dim, T_mb, max_seq_len):
    """Random vector generation.

    Args:
      - batch_size: size of the random vector
      - z_dim: dimension of random vector
      - T_mb: time information for the random vector
      - max_seq_len: maximum sequence length

    Returns:
      - Z_mb: generated random vector
    """
    Z_mb = list()
    for i in range(batch_size):
        temp = np.zeros([max_seq_len, z_dim])
        temp_Z = np.random.uniform(0.0, 1, [T_mb[i], z_dim])
        temp[: T_mb[i], :] = temp_Z
        Z_mb.append(temp)
    return torch.tensor(np.stack(Z_mb)).float()


def _weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0)
    elif classname.find("Conv") != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find("Norm") != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for name, param in m.named_parameters():
            if "weight_ih" in name:
                init.xavier_uniform_(param.data)
            elif "weight_hh" in name:
                init.orthogonal_(param.data)
            elif "bias" in name:
                param.data.fill_(0)



In [ ]:
class Encoder(nn.Module):
    """Embedding network between original feature space to latent space.

    Args:
      - input: input time-series features. (L, N, X) = (24, ?, 6)
      - h3: (num_layers, N, H). [3, ?, 24]

    Returns:
      - H: embeddings
    """

    def __init__(self, input_size, d_model, num_layers, num_heads):
        super(Encoder, self).__init__()


        self.encoder_proj = nn.Linear(
            input_size, d_model
        )
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            batch_first=True,
        )

        self.encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers,
            norm=nn.LayerNorm(),
            enable_nested_tensor=True,
            norm_first=True,
        )
        self.fc = nn.Linear(d_model, d_model)
        # в оригинале стремная инициализация весов
        self.apply(_weights_init)

    def forward(self, x):
        e_outputs, = self.encoder(self.encoder_proj(x))
        H = self.fc(e_outputs)
        return H


class Recovery(nn.Module):
    """Recovery network from latent space to original space.

    Args:
      - H: latent representation
      - T: input time information

    Returns:
      - X_tilde: recovered data
    """

    def __init__(self, input_size, hidden_rnn, num_layers):
        super(Recovery, self).__init__()
        self.rnn = nn.GRU(
            input_size=hidden_rnn,
            hidden_size=input_size,
            num_layers=num_layers,
            batch_first=True,
        )

        self.fc = nn.Linear(input_size, input_size)
        self.sigmoid = nn.Sigmoid()
        self.apply(_weights_init)

    def forward(self, input, sigmoid=True):
        r_outputs, _ = self.rnn(input)
        X_tilde = self.fc(r_outputs)
        if sigmoid:
            X_tilde = self.sigmoid(X_tilde)
        return X_tilde


class Generator(nn.Module):
    """Generator function: Generate time-series data in latent space.

    Args:
      - Z: random variables
      - T: input time information

    Returns:
      - E: generated embedding
    """

    def __init__(self, input_size, hidden_rnn, num_layers):
        super(Generator, self).__init__()
        self.rnn = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_rnn,
            num_layers=num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_rnn, hidden_rnn)
        self.sigmoid = nn.Sigmoid()
        self.apply(_weights_init)

    def forward(self, input, sigmoid=True):
        g_outputs, _ = self.rnn(input)
        E = self.fc(g_outputs)
        if sigmoid:
            E = self.sigmoid(E)
        return E


class Supervisor(nn.Module):
    """Generate next sequence using the previous sequence.

    Args:
      - H: latent representation
      - T: input time information

    Returns:
      - S: generated sequence based on the latent representations generated by the generator
    """

    def __init__(self, hidden_rnn, num_layers):
        super(Supervisor, self).__init__()
        self.rnn = nn.GRU(
            input_size=hidden_rnn,
            hidden_size=hidden_rnn,
            num_layers=num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_rnn, hidden_rnn)
        self.sigmoid = nn.Sigmoid()
        self.apply(_weights_init)

    def forward(self, input, sigmoid=True):
        s_outputs, _ = self.rnn(input)
        S = self.fc(s_outputs)
        if sigmoid:
            S = self.sigmoid(S)
        return S


class Discriminator(nn.Module):
    """Discriminate the original and synthetic time-series data.

    Args:
      - H: latent representation
      - T: input time information

    Returns:
      - Y_hat: classification results between original and synthetic time-series
    """

    def __init__(self, hidden_rnn, num_layers):
        super(Discriminator, self).__init__()
        self.rnn = nn.GRU(
            input_size=hidden_rnn,
            hidden_size=hidden_rnn,
            num_layers=num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_rnn, 2)
        self.apply(_weights_init)

    def forward(self, input):
        d_outputs, _ = self.rnn(input)
        Y_hat = self.fc(d_outputs)
        return Y_hat


class TG(nn.Module):
    def __init__(self, data_conf, model_conf):
        super().__init__()
        self.model_conf = model_conf
        self.data_conf = data_conf

        ### PROCESSORS ###
        self.processor = prp.FeatureProcessor(
            model_conf=model_conf, data_conf=data_conf
        )

        ### INPUT SIZE ###
        all_emb_size = self.model_conf.features_emb_dim * len(
            self.data_conf.features.embeddings
        )

        self.all_numeric_size = (
            len(self.data_conf.features.numeric_values)
            * self.model_conf.numeric_emb_size
        )

        self.input_dim = all_emb_size + self.all_numeric_size

        # Pos Encoding
        self.enc_pos_encoding = nn.Embedding(
                self.data_conf.test.max_seq_len + 1, self.model_conf.encoder_hidden
            )
        self.cls_token = nn.Parameter(torch.rand(self.model_conf.encoder_hidden))

        self.encoder = Encoder(
            input_size=self.input_dim,
            hidden_rnn=self.model_conf.timegan.rnn_hidden,
            num_layers=self.model_conf.timegan.num_layers,
        )
        self.decoder = Recovery(
            input_size=self.input_dim,
            hidden_rnn=self.model_conf.timegan.rnn_hidden,
            num_layers=self.model_conf.timegan.num_layers,
        )
        self.supervisor = Supervisor(
            hidden_rnn=self.model_conf.timegan.rnn_hidden,
            num_layers=self.model_conf.timegan.num_layers,
        )
        self.generator = Generator(
            input_size=self.input_dim,
            hidden_rnn=self.model_conf.timegan.rnn_hidden,
            num_layers=self.model_conf.timegan.num_layers,
        )
        self.discriminator = Discriminator(
            hidden_rnn=self.model_conf.timegan.rnn_hidden,
            num_layers=self.model_conf.timegan.num_layers,
        )

        self.gamma = self.model_conf.timegan.gamma

        # Predictors
        self.embedding_predictor = EmbeddingPredictor(
            model_conf=self.model_conf, data_conf=self.data_conf
        )
        self.numeric_projector = NumericalFeatureProjector(
            model_conf=self.model_conf, data_conf=self.data_conf
        )
        self.mse_fn = torch.nn.MSELoss(reduction="none")
        # self.ce_fn = torch.nn.CrossEntropyLoss(
        #     reduction="mean", ignore_index=0, label_smoothing=0.15
        # )

    def numerical_loss(self, pred, input_batch):
        # MSE
        total_mse_loss = 0
        for key, values in input_batch.payload.items():
            if key in self.processor.numeric_names:
                gt_val = values.float()
                gt_val = values.float()
                pred_val = pred[key].squeeze(-1)

                mse_loss = self.mse_fn(
                    gt_val,
                    pred_val,
                )
                mask = gt_val != 0
                masked_mse = mse_loss * mask
                total_mse_loss += (
                    masked_mse.sum(dim=1)  # / (mask != 0).sum(dim=1)
                ).mean()

        return total_mse_loss

    def e_loss(self, decoded, padded_batch):
        emb_dist = self.embedding_predictor(decoded)
        cross_entropy_losses = self.embedding_predictor.loss(emb_dist, padded_batch)
        total_ce_loss = torch.sum(
            torch.cat([value.unsqueeze(0) for _, value in cross_entropy_losses.items()])
        )

        mse_loss = self.numerical_loss(self.numeric_projector(decoded), padded_batch)

        return total_ce_loss + mse_loss

    def train_embedder(self, padded_batch):
        x, time_steps = self.processor(padded_batch)
        latens = self.encoder(x)
        decoded = self.decoder(latens)

        lens = padded_batch.seq_lens - 1
        global_hidden = latens[:, lens, :].diagonal().T

        total_loss = self.e_loss(decoded, padded_batch)

        return global_hidden, total_loss

    def train_generator(self, padded_batch):
        x, time_steps = self.processor(padded_batch)
        bs, l, d = x.size()
        Z = random_generator(bs, d, [l] * bs, l)
        gen_E = self.generator(Z.to(self.model_conf.device))

        gen_latens = self.supervisor(gen_E)
        latens = self.encoder(x)

        mse = F.mse_loss(latens, gen_latens, reduction="none").sum(dim=[1, 2]).mean()
        return mse

    def train_joint(self, padded_batch):
        x, time_steps = self.processor(padded_batch)
        bs, l, d = x.size()
        Z = random_generator(bs, d, [l] * bs, l)
        gen_latens = self.supervisor(self.generator(Z.to(self.model_conf.device)))
        latens = self.encoder(x)
        decoded = self.decoder(latens)
        gen_decoded = self.decoder(gen_latens)

        y_fake = self.discriminator(gen_latens)
        g_loss_u = (
            F.cross_entropy(
                y_fake.permute(0, 2, 1),
                torch.ones_like(y_fake)[:, :, 0].long(),
                reduction="none",
            )
            .sum(dim=1)
            .mean()
        )
        g_loss_s = (
            F.mse_loss(latens, gen_latens, reduction="none").sum(dim=[1, 2]).mean()
        )

        var_loss = torch.abs(
            torch.sqrt(torch.var(gen_decoded, dim=0) + 1e-6)
            - torch.sqrt(torch.var(x.detach(), dim=0) + 1e-6)
        ).mean()
        mean_loss = torch.abs(
            torch.mean(gen_decoded, dim=0) - torch.mean(x.detach(), dim=0)
        ).mean()
        g_loss_v = var_loss + mean_loss

        e_loss = self.e_loss(decoded, padded_batch)
        e_loss = 10 * torch.sqrt(e_loss)
        e_loss = e_loss + 0.1 * g_loss_s

        return g_loss_u, g_loss_s, g_loss_v, e_loss

    def train_discriminator(self, padded_batch):
        x, time_steps = self.processor(padded_batch)
        bs, l, d = x.size()
        Z = random_generator(bs, d, [l] * bs, l)
        e_gen = self.generator(Z.to(self.model_conf.device))
        gen_latens = self.supervisor(e_gen)
        latens = self.encoder(x)

        y_fake = self.discriminator(gen_latens)
        y_fake_e = self.discriminator(e_gen)
        y_real = self.discriminator(latens)
        D_loss_fake = (
            F.cross_entropy(
                y_fake.permute(0, 2, 1),
                torch.zeros_like(y_fake)[:, :, 0].long(),
                reduction="none",
            )
            .sum(dim=1)
            .mean()
        )
        D_loss_real = (
            F.cross_entropy(
                y_real.permute(0, 2, 1),
                torch.ones_like(y_real)[:, :, 0].long(),
                reduction="none",
            )
            .sum(dim=1)
            .mean()
        )
        D_loss_fake_e = (
            F.cross_entropy(
                y_fake_e.permute(0, 2, 1),
                torch.zeros_like(y_fake_e)[:, :, 0].long(),
                reduction="none",
            )
            .sum(dim=1)
            .mean()
        )
        D_loss = D_loss_real + D_loss_fake + self.gamma * D_loss_fake_e

        if D_loss.item() <= 0.15:
            D_loss = torch.tensor(0)

        return D_loss

    def reconstruct(self, padded_batch):
        x, time_steps = self.processor(padded_batch)
        latens = self.encoder(x)
        decoded = self.decoder(latens)

        pred = self.embedding_predictor(decoded)
        pred.update(self.numeric_projector(decoded))

        out = {"pred": pred, "time_steps": time_steps}
        return out

    def generate(self, padded_batch):
        x, time_steps = self.processor(padded_batch)
        bs, l, d = x.size()
        Z = random_generator(bs, d, [l] * bs, l)
        gen_latens = self.supervisor(self.generator(Z.to(self.model_conf.device)))
        gen_decoded = self.decoder(gen_latens)

        pred = self.embedding_predictor(gen_decoded)
        pred.update(self.numeric_projector(gen_decoded))

        out = {"pred": pred, "time_steps": time_steps}
        return out


In [2]:
data_conf = data_configs()
model_conf = model_configs()

In [3]:
train_loader, val_loader = create_data_loaders(data_conf, supervised=False)

Data shapes: train 8467, val 946, test 0


In [4]:
for batch in train_loader:
    break

In [5]:
tg = TG(model_conf=model_conf, data_conf=data_conf)

In [8]:
mse = tg.train_discriminator(batch[0])

tensor(420.7841, grad_fn=<AddBackward0>)


In [19]:
out = tg.generate(batch[0])

In [8]:
from src.models.TimeGan import random_generator

x, time_steps = tg.processor(batch[0])
bs, l, d = x.size()
Z = random_generator(bs, d, [l]*bs, l)
gen_latens = tg.supervisor(tg.generator(Z))
y_fake = tg.discriminator(gen_latens)

In [7]:
F.cross_entropy(y_fake.permute(0,2,1), torch.ones_like(y_fake).squeeze(-1).lo, reduction='none').size()

NameError: name 'y_fake' is not defined

In [11]:
y_fake.permute(0,2,1).size(), torch.ones_like(y_fake)[:,:,0].size()

(torch.Size([128, 2, 200]), torch.Size([128, 200]))

In [22]:
opt = torch.optim.Adam(
            list(tg.encoder.parameters()) + list(tg.decoder.parameters()), model_conf.lr, weight_decay=model_conf.weight_decay
        )

In [3]:
x = torch.rand(3, 5, 7) 
TG = TG(input_size=7, hidden_rnn=11, num_layers=2)

In [4]:
out = TG.train_discriminator(x)

tensor(-0.2320, grad_fn=<AddBackward0>)


AttributeError: type object 'TG' has no attribute 'encoder'

In [66]:
d_scores, gen_latens, decoded, x1 = net(x)

(torch.Size([3, 5, 7]), torch.Size([3, 5, 7]))

In [72]:
F.mse_loss(x1, decoded, reduction='none').sum(dim=[1,2]).mean()

tensor(14.1805, grad_fn=<MeanBackward0>)